# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [203]:
import numpy as np
import pandas as pd
import re
from scipy import stats

In [204]:
cities = pd.read_html('assets/wikipedia_data.html')[1]
cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
cities.drop(51, inplace=True)
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [205]:
big4 = 'NHL'

# Divide the cities in columns.
# e.g. 1 column with value | RangersIslandersDevils |
# gets converted to 3 columns | Rangers | Islanders | Devils |
teams = cities[big4].str.extract('([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)')

teams['Metropolitan area'] = cities['Metropolitan area']

# melt makes that a row that have many columns with one identifier, it converts all this columns to rows
# e.g. | NY | a | b | c | se convierte en 
#       | NY | a | 
#       | NY | b | 
#       | NY | c |
teams = pd.melt(teams, id_vars=['Metropolitan area'])

#teams = teams.replace('', np.nan)
#teams = teams[teams['value'].isnull() == False].reset_index()

# Filter the values that are null ('')
teams = teams[teams['value'] != ''].reset_index()

# get the population
teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
teams.rename(columns={'value': 'team'}, inplace=True)
teams = teams.astype({'Population': int})
teams.head()

,Metropolitan area,team,Population
0,New York City,Rangers,20153634
1,Los Angeles,Kings,13310447
2,San Francisco Bay Area,Sharks,6657982
3,Chicago,Blackhawks,9512999
4,Dallas–Fort Worth,Stars,7233323


In [206]:
nhl_df = pd.read_csv('assets/nhl.csv')
nhl_df = nhl_df[nhl_df['year'] == 2018]

nhl_df = nhl_df[['team', 'W', 'L']] 

def quitarfilas(row):
    if row['team'] == row['W']:
        row['good'] = False
    else:
        row['good'] = True
    return row
nhl_df = nhl_df.apply(quitarfilas, axis=1)

nhl_df = nhl_df[nhl_df['good'] == True][['team', 'W', 'L']]
#nhl_df = nhl_df.str.replace()
nhl_df = nhl_df.astype({'W': int, 'L': int})
nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)
nhl_df['team'] = nhl_df['team'].str.replace('\*', '', regex=True)
nhl_df.head()

,team,W,L,wl_ratio
1,Lightning,54,23,0.701299
2,Bruins,50,20,0.714286
3,Leafs,49,26,0.653333
4,Panthers,44,30,0.594595
5,Wings,30,39,0.434783


In [207]:
teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
merge = pd.merge(teams, nhl_df, how='left', on='team')
merge.head()

,Metropolitan area,team,Population,W,L,wl_ratio
0,New York City,Rangers,20153634,34,39,0.465753
1,Los Angeles,Kings,13310447,45,29,0.608108
2,San Francisco Bay Area,Sharks,6657982,45,27,0.625000
3,Chicago,Blackhawks,9512999,33,39,0.458333
4,Dallas–Fort Worth,Stars,7233323,42,32,0.567568


In [208]:
merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                               Population = ('Population', 'mean'))
merge.head()

,wl_ratio,Population
Metropolitan area,,
Boston,0.714286,4794447.0
Buffalo,0.357143,1132804.0
Calgary,0.513889,1392609.0
Chicago,0.458333,9512999.0
Columbus,0.600000,2041520.0


In [212]:
correlation = stats.pearsonr(merge['wl_ratio'], merge['Population'])
correlation, correlation[0]

(PearsonRResult(statistic=0.012486162921209907, pvalue=0.9497182859911791),
 0.012486162921209907)

In [216]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

#cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE

    # ------------------------------------
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
    cities.drop(51, inplace=True)
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)

    # -----------------------------------
    big4 = 'NHL'

    # Divide the cities in columns.
    # e.g. 1 column with value | RangersIslandersDevils |
    # gets converted to 3 columns | Rangers | Islanders | Devils |
    teams = cities[big4].str.extract('([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)')

    teams['Metropolitan area'] = cities['Metropolitan area']

    # melt makes that a row that have many columns with one identifier, it converts all this columns to rows
    # e.g. | NY | a | b | c | se convierte en 
    #       | NY | a | 
    #       | NY | b | 
    #       | NY | c |
    teams = pd.melt(teams, id_vars=['Metropolitan area'])

    #teams = teams.replace('', np.nan)
    #teams = teams[teams['value'].isnull() == False].reset_index()

    # Filter the values that are null ('')
    teams = teams[teams['value'] != ''].reset_index()

    # get the population
    teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
    teams.rename(columns={'value': 'team'}, inplace=True)
    teams = teams.astype({'Population': int})

    # ---------------------------------
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018]

    nhl_df = nhl_df[['team', 'W', 'L']] 

    def quitarfilas(row):
        if row['team'] == row['W']:
            row['good'] = False
        else:
            row['good'] = True
        return row
    nhl_df = nhl_df.apply(quitarfilas, axis=1)

    nhl_df = nhl_df[nhl_df['good'] == True][['team', 'W', 'L']]
    #nhl_df = nhl_df.str.replace()
    nhl_df = nhl_df.astype({'W': int, 'L': int})
    nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)
    nhl_df['team'] = nhl_df['team'].str.replace('\*', '', regex=True)

    # -----------------------------------
    teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
    merge = pd.merge(teams, nhl_df, how='left', on='team')

    merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                                Population = ('Population', 'mean'))
        
    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [14]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
cities.drop(51, inplace=True)
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [83]:
# -----------------------------------
big4 = 'NBA'

# Divide the cities in columns.
# e.g. 1 column with value | RangersIslandersDevils |
# gets converted to 3 columns | Rangers | Islanders | Devils |
teams = cities[big4].str.extract('([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')

teams['Metropolitan area'] = cities['Metropolitan area']

teams = pd.melt(teams, id_vars=['Metropolitan area'])

teams = teams[teams['value'] != ''].reset_index()

# get the population
teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
teams.rename(columns={'value': 'team'}, inplace=True)
teams = teams.astype({'Population': int})
teams.head()

,Metropolitan area,team,Population
0,New York City,Knicks,20153634
1,Los Angeles,Lakers,13310447
2,San Francisco Bay Area,Warriors,6657982
3,Chicago,Bulls,9512999
4,Dallas–Fort Worth,Mavericks,7233323


In [72]:
nhl_df = pd.read_csv('assets/nba.csv')
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df = nhl_df[['team', 'W', 'L']]

nhl_df = nhl_df.astype({'W': int, 'L': int})
nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)
nhl_df['team'] = nhl_df['team'].str.replace('\*', '', regex=True)
nhl_df['team'] = nhl_df['team'].str.replace('\\xa0\([\d]*\)', '', regex=True)
nhl_df.head()

,team,W,L,wl_ratio
0,Raptors,59,23,0.719512
1,Celtics,55,27,0.670732
2,76ers,52,30,0.634146
3,Cavaliers,50,32,0.609756
4,Pacers,48,34,0.585366


In [77]:
teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
merge = pd.merge(teams, nhl_df, how='left', on='team')
merge.head()

,Metropolitan area,team,Population,W,L,wl_ratio
0,New York City,Knicks,20153634,29,53,0.353659
1,Los Angeles,Lakers,13310447,35,47,0.426829
2,San Francisco Bay Area,Warriors,6657982,58,24,0.707317
3,Chicago,Bulls,9512999,27,55,0.329268
4,Dallas–Fort Worth,Mavericks,7233323,24,58,0.292683


In [81]:
merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                               Population = ('Population', 'mean'))
merge.head()

,wl_ratio,Population
Metropolitan area,,
Atlanta,0.292683,5789700.0
Boston,0.670732,4794447.0
Charlotte,0.439024,2474314.0
Chicago,0.329268,9512999.0
Cleveland,0.609756,2055612.0


In [82]:
population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as 

stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.17657160252844623

In [84]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    
    # ----------------------------------
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
    cities.drop(51, inplace=True)
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)

    # -----------------------------------
    big4 = 'NBA'

    # Divide the cities in columns.
    # e.g. 1 column with value | RangersIslandersDevils |
    # gets converted to 3 columns | Rangers | Islanders | Devils |
    teams = cities[big4].str.extract('([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')

    teams['Metropolitan area'] = cities['Metropolitan area']

    teams = pd.melt(teams, id_vars=['Metropolitan area'])

    teams = teams[teams['value'] != ''].reset_index()

    # get the population
    teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
    teams.rename(columns={'value': 'team'}, inplace=True)
    teams = teams.astype({'Population': int})

    # ------------------------------------
    nhl_df = pd.read_csv('assets/nba.csv')
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[['team', 'W', 'L']]

    nhl_df = nhl_df.astype({'W': int, 'L': int})
    nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)
    nhl_df['team'] = nhl_df['team'].str.replace('\*', '', regex=True)
    nhl_df['team'] = nhl_df['team'].str.replace('\\xa0\([\d]*\)', '', regex=True)

    # ------------------------------------------
    teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
    merge = pd.merge(teams, nhl_df, how='left', on='team')

    # ------------
    merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                                Population = ('Population', 'mean'))
    # -------------
    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as 
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844623

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [96]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
cities.drop(51, inplace=True)
cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [123]:
# -----------------------------------
big4 = 'MLB'

# Divide the cities in columns.
# e.g. 1 column with value | RangersIslandersDevils |
# gets converted to 3 columns | Rangers | Islanders | Devils |
teams = cities[big4].str.extract('([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')

teams['Metropolitan area'] = cities['Metropolitan area']

teams = pd.melt(teams, id_vars=['Metropolitan area'])

teams = teams[teams['value'] != ''].reset_index()


# get the population
teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
teams.rename(columns={'value': 'team'}, inplace=True)
teams = teams.astype({'Population': int})
teams['team'] = teams['team'].str.replace('\ Sox', 'Sox', regex=True)
teams.head()

,Metropolitan area,team,Population
0,New York City,Yankees,20153634
1,Los Angeles,Dodgers,13310447
2,San Francisco Bay Area,Giants,6657982
3,Chicago,Cubs,9512999
4,Dallas–Fort Worth,Rangers,7233323


In [124]:
# ------------------------------------
nhl_df = pd.read_csv('assets/mlb.csv')
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df = nhl_df[['team', 'W', 'L']]

nhl_df = nhl_df.astype({'W': int, 'L': int})
nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

nhl_df['team'] = nhl_df['team'].str.replace('\ Sox', 'Sox', regex=True)
nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)
# nhl_df['team'] = nhl_df['team'].str.replace('\*', '', regex=True)
# nhl_df['team'] = nhl_df['team'].str.replace('\\xa0\([\d]*\)', '', regex=True)
nhl_df.head()

,team,W,L,wl_ratio
0,RedSox,108,54,0.666667
1,Yankees,100,62,0.617284
2,Rays,90,72,0.555556
3,Jays,73,89,0.450617
4,Orioles,47,115,0.290123


In [126]:
# ------------------------------------------
teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
merge = pd.merge(teams, nhl_df, how='left', on='team')
merge.head()

,Metropolitan area,team,Population,W,L,wl_ratio
0,New York City,Yankees,20153634,100,62,0.617284
1,Los Angeles,Dodgers,13310447,92,71,0.564417
2,San Francisco Bay Area,Giants,6657982,73,89,0.450617
3,Chicago,Cubs,9512999,95,68,0.582822
4,Dallas–Fort Worth,Rangers,7233323,67,95,0.413580


In [127]:
# ------------
merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                            Population = ('Population', 'mean'))
# -------------
population_by_region = merge['Population'] # pass in metropolitan area population from cities
win_loss_by_region = merge['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as 

stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.15027698302669307

In [128]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    # -----------------------
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities.drop(['Country', 'Pop.rank', 'B4', 'B6', 'MLS', 'CFL'], axis=1, inplace=True)
    cities.drop(51, inplace=True)
    cities.rename(columns={'Population (2016 est.)[8]': 'Population'}, inplace=True)
    cities.replace(to_replace='(\[.*\]$)', value='', regex=True, inplace=True)

    # -------------------------
    big4 = 'MLB'

    # Divide the cities in columns.
    # e.g. 1 column with value | RangersIslandersDevils |
    # gets converted to 3 columns | Rangers | Islanders | Devils |
    teams = cities[big4].str.extract('([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)([A-Z0-9]{0,2}[a-z0-9]*\ [A-Z0-9]{0,2}[a-z0-9]*|[A-Z0-9]{0,2}[a-z0-9]*)')

    teams['Metropolitan area'] = cities['Metropolitan area']

    teams = pd.melt(teams, id_vars=['Metropolitan area'])

    teams = teams[teams['value'] != ''].reset_index()


    # get the population
    teams = pd.merge(teams, cities, how='left', on='Metropolitan area')[['Metropolitan area', 'value', 'Population']]
    teams.rename(columns={'value': 'team'}, inplace=True)
    teams = teams.astype({'Population': int})
    teams['team'] = teams['team'].str.replace('\ Sox', 'Sox', regex=True)

    # ------------------------------------
    nhl_df = pd.read_csv('assets/mlb.csv')
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df = nhl_df[['team', 'W', 'L']]

    nhl_df = nhl_df.astype({'W': int, 'L': int})
    nhl_df['wl_ratio'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    nhl_df['team'] = nhl_df['team'].str.replace('\ Sox', 'Sox', regex=True)
    nhl_df['team'] = nhl_df['team'].str.replace('[\w.]*\ ', '', regex=True)

    # ------------------------------------------
    teams['team'] = teams['team'].str.replace('[\w ]*\ ', '', regex=True)
    merge = pd.merge(teams, nhl_df, how='left', on='team')

    # ------------
    merge = merge.groupby('Metropolitan area').agg(wl_ratio = ('wl_ratio', 'mean'),
                                                Population = ('Population', 'mean'))
    # -------------
    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['wl_ratio'] # pass in win/loss ratio from nhl_df in the same order as

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values